### Cuidados!
 
1. Whatsapp não gosta de nenhum tipo de automação, se mandar para muitas pessoas pode ser bloqueio.
2. Isso pode dar merda
3. Isso não é o uso da API oficial do Whatsapp, o próprio whatsapp tem uma API oficial. Se o seu objetivo é fazer envio em massa ou criar aqueles robozinhos que respondem automaticamente no whatsapp, então use a API oficial
4. Meu objetivo é 100% educacional

### Dito isso, bora automatizar o envio de mensagens no Whatsapp

- Vamos usar o Selenium (vídeo da configuração na descrição)
- Temos 1 alternativa muito boa:
- Usar o https://web.whatsapp.com/send?phone=numero&text=texto (mais fácil, mais seguro, mas mais demorado)


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

servico = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument(r'user-data-dir=C:\Users\Pichau\AppData\Local\Google\Chrome\User Data\Profile Selenium')
navegador = webdriver.Chrome(service=servico, options = options)

navegador.get("https://web.whatsapp.com")

# esperar a tela do whatsapp carregar
while len(navegador.find_elements(By.ID, 'side')) < 1: # -> lista for vazia -> que o elemento não existe ainda
    time.sleep(1)
time.sleep(2) # só uma garantia

In [7]:
# o whatsapp já carregou
import pandas as pd

tabela = pd.read_excel("Envios.xlsx")
display(tabela) # tem também uma coluna telefone dentro da tabela

,nome,mensagem,arquivo,telefone
0,Diego Hashtag,"E aí, fulano, tudo certo?",N,NaN
1,Alon Hashtag,"Coe fulano, bora surfar?",N,NaN
2,Lira Hashtag,"Viajou, fulano. Olha isso:",pythonimpressionador.png,NaN


In [6]:
import urllib
import time
import os

### Sempre preste atenção quando for fazer esse codigo é ficar de olho nos elementos, pois o whatsapp sempre muda os elementos do site

for linha in tabela.index:
    # enviar uma mensagem para a pessoa
    nome = tabela.loc[linha, "nome"]
    mensagem = tabela.loc[linha, "mensagem"]
    arquivo = tabela.loc[linha, "arquivo"]
    telefone = tabela.loc[linha, "telefone"]
    
    texto = mensagem.replace("fulano", nome)
    texto = urllib.parse.quote(texto)

    # enviar a mensagem
    link = f"https://web.whatsapp.com/send?phone={telefone}&text={texto}"
    
    navegador.get(link)
    # esperar a tela do whatsapp carregar -> espera um elemento que só existe na tela já carregada aparecer
    while len(navegador.find_elements(By.ID, 'side')) < 1: # -> lista for vazia -> que o elemento não existe ainda
        time.sleep(1)
    time.sleep(2) # só uma garantia
    
    # você tem que verificar se o número é inválido
    if len(navegador.find_elements(By.XPATH, '//*[@id="app"]/div/span[2]/div/span/div/div/div/div/div/div[1]')) < 1:
        # enviar a mensagem
        navegador.find_element(By.XPATH, '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[2]/button/span').click()
        
        if arquivo != "N": #enviar arquivo 
            caminho_completo = os.path.abspath(f"arquivos/{arquivo}")
            navegador.find_element(By.XPATH, 
                                   '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/div/span').click() #clicar no botao de anexo 
            navegador.find_element(By.XPATH, 
                                   '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/span/div/div/ul/li[4]/button/input').send_keys(caminho_completo) #preencher informação dentro do input('logo de documento')
            time.sleep(2)
            navegador.find_element(By.XPATH, 
                                   '//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/div/div[2]/div[2]/div/div').click()
            
        time.sleep(5)

KeyboardInterrupt: 